## Task 1

First look up the coordinates of the *gyrA* gene on NCBI, these are: 1051396 to 1054146, which in python become 1051395 to 1054145 (numbering from zero).

Now read in the the super gonorrhoea mapped concensus fasta file:

In [1]:
from Bio import SeqIO
super_gc_mapped_fa = SeqIO.read('cdt-tutorial/super_gonorrhoea/super_gc_mapped.fa', 'fasta')

Extract the gene of interest and convert it to amino acid sequence:

In [2]:
super_gc_GyrA = super_gc_mapped_fa[1051395:1054146].translate()
print(super_gc_GyrA)

ID: <unknown id>
Name: <unknown name>
Description: <unknown description>
Number of features: 0
Seq('MTDATIRHDHKFALETLPVSLEDEMRKSYLDYAMSVIVGRALPDVRDGLKPVHR...EN*', HasStopCodon(ExtendedIUPACProtein(), '*'))


We can now extract the amino acid at position 91 (numbered 90 in python) in the protein:

In [3]:
super_gc_GyrA[90]

'F'

So we confirm evidence of the S91F mutation.

We now want to compare this to the reference sequence, first read in the reference, and then translate as before:

In [4]:
ref_fa = SeqIO.read('cdt-tutorial/super_gonorrhoea/reference.fa', 'fasta')
ref_GyrA = ref_fa[1051395:1054146].translate()

Now compare the two amino acid strings, e.g.:

In [5]:
for i, (aa1, aa2) in enumerate(zip(super_gc_GyrA, ref_GyrA)):
    if aa1!=aa2:
        print(i+1, aa1, aa2)

95 A G


You can see that the two sequences also differ at position 95. The wildtype position here is D, so the reference contains one mutation D95G and the super gonorrhoea another D95A

## Task 2
First let's create the BLAST database needed:

In [3]:
import os
cmd = 'makeblastdb -dbtype nucl -in cdt-tutorial/super_gonorrhoea/super_gc_contigs.fa'
os.system(cmd)

0

Check the expected output ending super_gc_contigs.fa.n*  has been generated:

In [7]:
os.listdir('cdt-tutorial/super_gonorrhoea/')

['23s_reference.fa',
 'penA_alleles.fa',
 'reference.fa',
 'super_gc_contigs.fa',
 'super_gc_mapped.fa',
 'tetM.fa',
 'super_gc_contigs.fa.ndb',
 'super_gc_contigs.fa.ntf',
 'super_gc_contigs.fa.nin',
 'super_gc_contigs.fa.nhr',
 'super_gc_contigs.fa.nsq',
 'super_gc_contigs.fa.not',
 'super_gc_contigs.fa.nto']

Now run blast using the code snippet provided:

In [4]:
from Bio.Blast import NCBIXML
from Bio.Blast.Applications import NcbiblastnCommandline
from io import StringIO

os.chdir('cdt-tutorial/super_gonorrhoea/')

geneFile = 'tetM.fa'
assemblyFile = 'super_gc_contigs.fa'

cline = NcbiblastnCommandline( query=geneFile, 
         db=assemblyFile, 
         evalue=0.01, 
         outfmt=5
         )
stdout, stderr = cline()
fp = StringIO( stdout )
blast_records = [r for r in NCBIXML.parse( fp )]

You now need to parse the output to find what you need:

In [26]:
for record in blast_records:
    print('%s has %s alignments\n'%(record.query, len(record.alignments)))
    if len(record.alignments)>0:
        hits = []
        for alignment in record.alignments:
            for hsp in alignment.hsps:
                print('tetM length: %s'%len(hsp.query))
                print('Alignment length: %s'%hsp.align_length)
                print('Number of identities: %s'%hsp.identities)
                print('Number of gaps: %s'%hsp.gaps)

tetM has 1 alignments

tetM length: 1932
Alignment length: 1932
Number of identities: 1932
Number of gaps: 0


You can see we have found a perfect match.

## Task 3

Now search for a match in the *penA* alleles:

In [27]:
geneFile = 'penA_alleles.fa'
assemblyFile = 'super_gc_contigs.fa'

cline = NcbiblastnCommandline( query=geneFile, 
         db=assemblyFile, 
         evalue=0.01, 
         outfmt=5
         )
stdout, stderr = cline()
fp = StringIO( stdout )
blast_records = [r for r in NCBIXML.parse( fp )]

If we parse the results of the blast search as before we have lots of hits

In [42]:
for record in blast_records:
    print('%s has %s alignments'%(record.query, len(record.alignments)))
    if len(record.alignments)>0:
        hits = []
        for alignment in record.alignments:
            for hsp in alignment.hsps:
                print('penA length: %s'%len(hsp.query))
                print('Alignment length: %s'%hsp.align_length)
                print('Number of identities: %s'%hsp.identities)
                print('Number of gaps: %s\n'%hsp.gaps)

penA_1.001 has 1 alignments
penA length: 1756
Alignment length: 1756
Number of identities: 1585
Number of gaps: 14

penA_1.002 has 1 alignments
penA length: 1756
Alignment length: 1756
Number of identities: 1583
Number of gaps: 14

penA_1.003 has 1 alignments
penA length: 1756
Alignment length: 1756
Number of identities: 1584
Number of gaps: 14

penA_1.004 has 1 alignments
penA length: 1756
Alignment length: 1756
Number of identities: 1584
Number of gaps: 14

penA_1.005 has 1 alignments
penA length: 1756
Alignment length: 1756
Number of identities: 1584
Number of gaps: 14

penA_1.006 has 1 alignments
penA length: 1756
Alignment length: 1756
Number of identities: 1583
Number of gaps: 14

penA_1.007 has 1 alignments
penA length: 1756
Alignment length: 1756
Number of identities: 1583
Number of gaps: 14

penA_1.008 has 1 alignments
penA length: 1756
Alignment length: 1756
Number of identities: 1585
Number of gaps: 14

penA_1.009 has 1 alignments
penA length: 1756
Alignment length: 1756
Num

We now need to find the best hit, e.g. the one with the highest number of identities:

In [48]:
hsp_list = []
for record in blast_records:
    if len(record.alignments)>0:
        hits = []
        for alignment in record.alignments:
            for hsp in alignment.hsps:
                hsp_summary = [record.query, hsp.align_length, hsp.identities, hsp.gaps, hsp.sbjct]
                hsp_list.append(hsp_summary)

sorted_hsps = sorted(hsp_list, key=lambda hsp_list: hsp_list[2], reverse=True)
print(sorted_hsps[0])

['penA_60.001', 1749, 1749, 0, 'ATGTTGATTAAAAGCGAATATAAGCCCCGGATGCTGCCCAAAGAAGAGCAGGTCAAAAAGCCGATGACCAGTAACGGACGGATTAGCTTCGTCCTGATGGCAATGGCGGTCTTGTTTGCCTGTCTGATTGCCCGCGGGCTGTATCTGCAGACGGTAACGTATAACTTTTTGAAAGAACAGGGCGACAACCGGATTGTGCGGACTCAAGCATTGCCGGCTACACGCGGTACGGTTTCGGACCGGAACGGTGCGGTTTTGGCGTTGAGCGCGCCGACGGAGTCCCTGTTTGCCGTGCCTAAAGATATGAAGGAAATGCCGTCTGCCGCCCAATTGGAACGCCTGTCCGAGCTTGTCGATGTGCCGGTCGATGTTTTGAGGAACAAACTCGAACAGAAAGGCAAGTCGTTTATTTGGATCAAGCGGCAGCTCGATCCCAAGGTTGCCGAAGAGGTCAAAGCCTTGGGTTTGGAAAACTTTGTATTTGAAAAAGAATTAAAACGCCATTACCCGATGGGCAACCTGTTTGCACACGTCATCGGATTTACCGATATTGACGGCAAAGGTCAGGAAGGTTTGGAACTTTCGCTTGAAGACAGCCTGTATGGCGAAGACGGCGCGGAAGTTGTTTTGCGGGACCGGCAGGGCAATATTGTGGACAGCTTGGACTCCCCGCGCAATAAAGCACCGCAAAACGGCAAAGACATCATCCTTTCCCTCGATCAGAGGATTCAGACCTTGGCCTATGAAGAGTTGAACAAGGCGGTCGAATACCATCAGGCAAAAGCCGGAACGGTGGTGGTTTTGGATGCCCGCACGGGGGAAATCCTCGCCTTGGCCAATACGCCCGCCTACGATCCCAACAGACCCGGCCGGGCAGACAGCGAACAGCGGCGCAACCGTGCCGTTACCGACATGATCGAACCTGGTTCTGTCATGAAGCCGTTTACCATTGCCAAAGCATTGGATTC

The top his is *penA* 60.001, and this an exact match. We now want to analyse the amino acids in this sequence at positions 311 316 and 483. First convert the sequence matched back to a Biopython sequence object

In [51]:
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

top_hit = sorted_hsps[0]
seq = SeqRecord(Seq(top_hit[4]), name=top_hit[0])
print(seq)

ID: <unknown id>
Name: penA_60.001
Description: <unknown description>
Number of features: 0
Seq('ATGTTGATTAAAAGCGAATATAAGCCCCGGATGCTGCCCAAAGAAGAGCAGGTC...TAA')


In [53]:
protein = seq.translate()
print(protein[310], protein[315], protein[482])

V T S


You can see that A311V and T483S are present, but T316P. This is sufficient to confer ceftriaxone resistance.

## Task 4

We need to index the 23S gene reference:

In [54]:
cmd = 'bwa index 23s_reference.fa'
os.system(cmd)

0

Check for the index files ending 23s_reference.fa.*

In [56]:
os.listdir('.')

['23s_reference.fa',
 'penA_alleles.fa',
 'reference.fa',
 'super_gc_contigs.fa',
 'super_gc_mapped.fa',
 'tetM.fa',
 'super_gc_contigs.fa.ndb',
 'super_gc_contigs.fa.ntf',
 'super_gc_contigs.fa.nin',
 'super_gc_contigs.fa.nhr',
 'super_gc_contigs.fa.nsq',
 'super_gc_contigs.fa.not',
 'super_gc_contigs.fa.nto',
 '23s_reference.fa.pac',
 '23s_reference.fa.ann',
 '23s_reference.fa.amb',
 '23s_reference.fa.bwt',
 '23s_reference.fa.sa']

We now need to map the raw sequence read files to the new reference we have created. This will take several minutes to run, wait for the output below to include a zero before proceeding...

In [58]:
ref = '23s_reference.fa'
fq1 = '/cdtshared/wgs_practical/ERR2560139_1.fastq.gz'
fq2 = '/cdtshared/wgs_practical/ERR2560139_2.fastq.gz'
out_sam = '23s_mapped.sam'
cmd = 'bwa mem -t 4 %s %s %s > %s'%(ref, fq1, fq2, out_sam)
print(cmd)
os.system(cmd)

bwa mem -t 4 23s_reference.fa /cdtshared/wgs_practical/ERR2560139_1.fastq.gz /cdtshared/wgs_practical/ERR2560139_2.fastq.gz > 23s_mapped.sam


0

Now put only the mapped reads into their own bam file and index it:

In [60]:
out_bam = '23s_mapped.bam'
cmd = 'samtools view -uS -F 4 %s | samtools sort - -o %s'%(out_sam, out_bam)
print(cmd)
os.system(cmd)

cmd = 'samtools index %s'%out_bam
print(cmd)
os.system(cmd)

samtools view -uS -F 4 23s_mapped.sam | samtools sort - -o 23s_mapped.bam
samtools index 23s_mapped.bam


0

Now use pysam to analyse the mapped bam file, here we can also filter the reads based on quality, but this is an extra that is not part of the practical as written:

In [62]:
import pysam
samfile = pysam.Samfile(out_bam, "rb")

base2045 = []
base2597 = []

for pileupcolumn in samfile.pileup( '23s_rna', 2000, 2600):
    for pileupread in pileupcolumn.pileups:
        if pileupcolumn.pos == 2044 and pileupread.query_position != None:
            base = pileupread.alignment.query_sequence[pileupread.query_position]
            quality = ord(pileupread.alignment.qual[pileupread.query_position])
            if quality-33>=30:
                base2045.append(base)
        elif pileupcolumn.pos == 2596 and pileupread.query_position != None:
            base = pileupread.alignment.query_sequence[pileupread.query_position]
            quality = ord(pileupread.alignment.qual[pileupread.query_position])
            if quality-33>=30:
                base2597.append(base)

#check for base counts at base 2045, i.e. for evidence of A2059G
a = len([b for b in base2045 if b=='A'])
c = len([b for b in base2045 if b=='C'])
g = len([b for b in base2045 if b=='G'])
t = len([b for b in base2045 if b=='T'])

print(a, c, g, t)

0 1 5021 0


We find evidence of 4 copies of the A2059G mutation as nearly all the high quality bases are G. This will confer high level resistance to azithromycin.

We could do the same for C2611T:

In [63]:
a = len([b for b in base2597 if b=='A'])
c = len([b for b in base2597 if b=='C'])
g = len([b for b in base2597 if b=='G'])
t = len([b for b in base2597 if b=='T'])

print(a, c, g, t)

3 5342 3 1


Here we see there is only evidence of the wildtype. However, we have already explained the azithromycin resistance with the 4 copies of the A2059G mutation.

## Task 5
Well done you've made it to the final task. There are lots of ways to answer this task. I'll just provide an potential example, but your approach may well be better!

Read in the data as two pandas dataframes and just look at the first few rows to get a feel for the data

In [5]:
import pandas as pd
os.chdir('../hospital_transmission/')

samples = pd.read_csv('sample_list.csv')
snps = pd.read_csv('pairwise_snps.csv')

In [6]:
samples[0:10]

,collectdt,site,spec_num,rt,guid,id,fecal_tox,toxigenic
0,2013-01-19,8,1,20,8300636c-ba5e-4cce-9392-c1f2a873c6df,8300636c,NEG,1
1,2013-03-17,8,2,78,cf3a6a18-68e6-4ec0-a2ca-7012cade9be1,cf3a6a18,NEG,1
2,2013-01-14,8,3,39,85e8cf63-28ae-4d26-b639-4b8857838793,85e8cf63,NEG,1
3,2013-01-25,8,4,18,2658e7aa-768b-4975-8642-098ae9e40397,2658e7aa,POS,1
4,2013-02-07,8,5,14,f3ad5cb3-e94f-4e97-9ee9-ebeaf62742d0,f3ad5cb3,POS,0
5,2013-03-12,8,6,2,0da19dd8-ce8c-4669-ba7a-47ac62944f96,0da19dd8,NEG,1
6,2013-03-13,8,7,2,ee2c5fe7-fccf-4668-8ef4-5a8243006a0a,ee2c5fe7,NEG,1
7,2013-01-13,8,8,2,9e79d0bf-a55b-49ba-a138-5ebd48e52f55,9e79d0bf,NEG,1
8,2013-01-17,8,9,705,3d2d3cf4-cdfb-45c2-abed-7baf20f247b6,3d2d3cf4,POS,1
9,2013-01-10,8,10,15,1fc337a2-cf18-4a65-8937-34c1db315cae,1fc337a2,NEG,1


We want to use the fecal toxin column later setting the EQ (equivocal values) to be POS (positive) for the purposes of simplifying this analysis.

In [7]:
samples.groupby('fecal_tox').count()

,collectdt,site,spec_num,rt,guid,id,toxigenic
fecal_tox,,,,,,,
EQ,10,10,10,10,10,10,10
NEG,510,510,510,510,510,510,510
POS,453,453,453,453,453,453,453


In [8]:
samples.loc[samples['fecal_tox'] == 'EQ', 'fecal_tox'] = 'POS'
samples.groupby('fecal_tox').count()

,collectdt,site,spec_num,rt,guid,id,toxigenic
fecal_tox,,,,,,,
NEG,510,510,510,510,510,510,510
POS,463,463,463,463,463,463,463


In [9]:
snps[0:10]

,id1,id2,snps
0,2658e7aa,8300636c,14438
1,0da19dd8,8300636c,11560
2,3d2d3cf4,8300636c,29
3,1fc337a2,8300636c,13376
4,34096524,8300636c,12077
5,419fff1f,8300636c,96698
6,3f534ab6,8300636c,12263
7,3007bd9d,8300636c,100244
8,17783a9f,8300636c,12872
9,2a781b87,8300636c,11147


We want to ignore the non-toxigenic cases, so need to create a new list of samples:

In [10]:
print(len(samples))
toxigenic_samples = samples[samples['toxigenic']==1]
print(len(toxigenic_samples))

973
853


You can see that we have removed 120 non-toxigen samples, leave 853 to work with.

### Define the amount of within hospital transmission
We now want to define which hospital has the most transmission. We could define this for example as the proportion of cases with another earlier case in the dataset within ≤2 SNPs.

Let's first reduce the snps table to only include entries with ≤2 SNPs:

In [12]:
print(len(snps))
snps2 = snps[snps['snps']<=2]
print(len(snps2))

472878
556


To enable matching in either direction we now need to create a duplicate of the snps2 dataframe, but with the id1 and id2 reversed:

In [13]:
snps2_rev = snps2[['id2', 'id1', 'snps']]
snps2_rev.columns = ('id1', 'id2', 'snps') #rename columns so that are reversed when joined
snps2_all = pd.concat([snps2, snps2_rev], sort=False)
len(snps2_all.drop_duplicates())

1112

Now we need to merge the shorter SNPs table with the sample data, this will end up with fewer rows as were are only including the toxigenic cases:

In [14]:
j1 = pd.merge(snps2_all, toxigenic_samples, how='inner', left_on='id1', right_on='id')
merged = pd.merge(j1, toxigenic_samples, how='inner', left_on='id2', right_on='id', suffixes=('_1', '_2'))
print(len(merged))

1012


In [15]:
merged

,id1,id2,snps,collectdt_1,site_1,spec_num_1,rt_1,guid_1,id_1,fecal_tox_1,toxigenic_1,collectdt_2,site_2,spec_num_2,rt_2,guid_2,id_2,fecal_tox_2,toxigenic_2
0,419fff1f,cf3a6a18,1,2013-01-27,8,12,78,419fff1f-7db1-4430-9508-d12c6e052702,419fff1f,POS,1,2013-03-17,8,2,78,cf3a6a18-68e6-4ec0-a2ca-7012cade9be1,cf3a6a18,NEG,1
1,594fa74f,88072c26,2,2013-10-10,9,16,5,594fa74f-13ca-420f-a385-3dc8fdf70e34,594fa74f,POS,1,2013-02-05,8,14,13,88072c26-3f5d-471a-b917-9933c7791936,88072c26,POS,1
2,9e801c22,88072c26,2,2013-07-18,d,11,27,9e801c22-a02a-488f-9da5-4d845406e28c,9e801c22,POS,1,2013-02-05,8,14,13,88072c26-3f5d-471a-b917-9933c7791936,88072c26,POS,1
3,b1297db0,88072c26,2,2013-09-10,9,4,5,b1297db0-212b-47ba-8976-6e14e867af43,b1297db0,POS,1,2013-02-05,8,14,13,88072c26-3f5d-471a-b917-9933c7791936,88072c26,POS,1
4,594fa74f,b1297db0,0,2013-10-10,9,16,5,594fa74f-13ca-420f-a385-3dc8fdf70e34,594fa74f,POS,1,2013-09-10,9,4,5,b1297db0-212b-47ba-8976-6e14e867af43,b1297db0,POS,1
5,88072c26,b1297db0,2,2013-02-05,8,14,13,88072c26-3f5d-471a-b917-9933c7791936,88072c26,POS,1,2013-09-10,9,4,5,b1297db0-212b-47ba-8976-6e14e867af43,b1297db0,POS,1
6,9e801c22,b1297db0,0,2013-07-18,d,11,27,9e801c22-a02a-488f-9da5-4d845406e28c,9e801c22,POS,1,2013-09-10,9,4,5,b1297db0-212b-47ba-8976-6e14e867af43,b1297db0,POS,1
7,594fa74f,9e801c22,0,2013-10-10,9,16,5,594fa74f-13ca-420f-a385-3dc8fdf70e34,594fa74f,POS,1,2013-07-18,d,11,27,9e801c22-a02a-488f-9da5-4d845406e28c,9e801c22,POS,1
8,88072c26,9e801c22,2,2013-02-05,8,14,13,88072c26-3f5d-471a-b917-9933c7791936,88072c26,POS,1,2013-07-18,d,11,27,9e801c22-a02a-488f-9da5-4d845406e28c,9e801c22,POS,1
9,b1297db0,9e801c22,0,2013-09-10,9,4,5,b1297db0-212b-47ba-8976-6e14e867af43,b1297db0,POS,1,2013-07-18,d,11,27,9e801c22-a02a-488f-9da5-4d845406e28c,9e801c22,POS,1


We can now filter this down to ask how many of the pairs have an earlier case as case 1:

In [16]:
merged_ordered = merged[merged['collectdt_1']<merged['collectdt_2']]
len(merged_ordered)

489

Now restrict to only pairs in the same hospital:

In [17]:
merged_hospital = merged_ordered[merged_ordered['site_1']==merged_ordered['site_2']]
len(merged_hospital)

271

Now get a count of the number of cases at each hospital with a prior case within ≤ 2 SNPs, and compare this to the total number of cases. We need to only count one link per recipient case.

In [25]:
links = merged_hospital.groupby('site_1').id_2.nunique()
df_links = links.rename_axis('site').reset_index(name='links')
df_links

,site,links
0,1,34
1,8,15
2,9,47
3,b,36
4,c,21
5,d,13


In [26]:
cases = toxigenic_samples.groupby('site').count()['id']
df_cases = cases.rename_axis('site').reset_index(name='cases')
df_cases

,site,cases
0,1,175
1,8,136
2,9,183
3,b,162
4,c,110
5,d,87


In [27]:
summary = pd.merge(df_cases, df_links)
summary['pct_liinked'] = round(100*summary['links'] / summary['cases'],1)
summary

,site,cases,links,pct_liinked
0,1,175,34,19.4
1,8,136,15,11.0
2,9,183,47,25.7
3,b,162,36,22.2
4,c,110,21,19.1
5,d,87,13,14.9


Site 9 has the most transmission, and site 8 the least using this metric. In the paper we only consider recipients occuring after the first 90 days of the study at each hospital, you could considered trying this here.

### Define which cases are more infectious

We will use the same dataframe merged_hospital, but group by the fecal toxin column for the source case, i.e. the earlier case, case 1, and count the number of unique second cases linked to each fecal toxin type.

In [28]:
tmp = merged_hospital.groupby('fecal_tox_1').id_2.nunique()
links_tox = tmp.rename_axis('fecal_tox').reset_index(name='links')
links_tox

,fecal_tox,links
0,NEG,105
1,POS,92


In [29]:
tmp = toxigenic_samples.groupby('fecal_tox').count()['id']
cases_tox = tmp.rename_axis('fecal_tox').reset_index(name='cases')
cases_tox

,fecal_tox,cases
0,NEG,405
1,POS,448


In [30]:
summary_tox = pd.merge(cases_tox, links_tox)
summary_tox['pct_liinked'] = round(100*summary_tox['links'] / summary_tox['cases'],1)
summary_tox

,fecal_tox,cases,links,pct_liinked
0,NEG,405,105,25.9
1,POS,448,92,20.5


Potentially rates of linkage to a fecal toxin positive case are higher. However you could improve on this by considering the number of unique cases linked to a fecal toxin positive case only, a fecal toxin negative case only, or both. You are probably too short on time to do this today - but you could look at what we do in the paper if you have time.

### Transmission between hospitals

You know the answer is yes, from the above. But here is a table with possible links, there are 218. Is this suprising?

In [24]:
merged_ordered[merged_ordered['site_1']!=merged_ordered['site_2']]

,id1,id2,snps,collectdt_1,site_1,spec_num_1,rt_1,guid_1,id_1,fecal_tox_1,toxigenic_1,collectdt_2,site_2,spec_num_2,rt_2,guid_2,id_2,fecal_tox_2,toxigenic_2
5,88072c26,b1297db0,2,2013-02-05,8,14,13,88072c26-3f5d-471a-b917-9933c7791936,88072c26,POS,1,2013-09-10,9,4,5,b1297db0-212b-47ba-8976-6e14e867af43,b1297db0,POS,1
6,9e801c22,b1297db0,0,2013-07-18,d,11,27,9e801c22-a02a-488f-9da5-4d845406e28c,9e801c22,POS,1,2013-09-10,9,4,5,b1297db0-212b-47ba-8976-6e14e867af43,b1297db0,POS,1
8,88072c26,9e801c22,2,2013-02-05,8,14,13,88072c26-3f5d-471a-b917-9933c7791936,88072c26,POS,1,2013-07-18,d,11,27,9e801c22-a02a-488f-9da5-4d845406e28c,9e801c22,POS,1
11,88072c26,594fa74f,2,2013-02-05,8,14,13,88072c26-3f5d-471a-b917-9933c7791936,88072c26,POS,1,2013-10-10,9,16,5,594fa74f-13ca-420f-a385-3dc8fdf70e34,594fa74f,POS,1
12,9e801c22,594fa74f,0,2013-07-18,d,11,27,9e801c22-a02a-488f-9da5-4d845406e28c,9e801c22,POS,1,2013-10-10,9,16,5,594fa74f-13ca-420f-a385-3dc8fdf70e34,594fa74f,POS,1
16,73740cb9,4b17cf00,2,2013-10-28,9,22,78,73740cb9-1696-408d-8ca9-1b9d848346da,73740cb9,POS,1,2013-11-26,1,181,78,4b17cf00-c43e-452b-bbb4-ea7937cb519e,4b17cf00,NEG,1
18,9476308f,4b17cf00,2,2013-09-20,9,9,78,9476308f-3276-4fcb-a84b-628dfa5320c7,9476308f,POS,1,2013-11-26,1,181,78,4b17cf00-c43e-452b-bbb4-ea7937cb519e,4b17cf00,NEG,1
21,4b17cf00,369774fe,2,2013-11-26,1,181,78,4b17cf00-c43e-452b-bbb4-ea7937cb519e,4b17cf00,NEG,1,2014-03-14,b,90,78,369774fe-7297-44f9-91de-6e9863e34759,369774fe,NEG,1
22,3ff390d4,369774fe,1,2014-01-15,c,26,78,3ff390d4-8a03-47f1-8b80-189faa964e93,3ff390d4,POS,1,2014-03-14,b,90,78,369774fe-7297-44f9-91de-6e9863e34759,369774fe,NEG,1
23,3ec5feb4,369774fe,2,2013-09-04,c,55,78,3ec5feb4-a78c-4dd7-be2b-f764840f5722,3ec5feb4,NEG,1,2014-03-14,b,90,78,369774fe-7297-44f9-91de-6e9863e34759,369774fe,NEG,1
